In [1]:
import numpy as np
import mxnet as mx
from collections import namedtuple, Counter
from unidecode import unidecode
from itertools import groupby
from mxnet.io import DataIter
from random import shuffle

import deepdish as dd

import operator
import pickle
import re
import warnings

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-29 23:43:20.196636. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [2]:
# Get rid of annoying Python deprecation warnings from built-in JSON encoder
warnings.filterwarnings("ignore", category=DeprecationWarning)   

In [3]:
# Decode text as UTF-8
# Remove diacritical signs and convert to Latin alphabet
# Separate punctuation as separate "words"
def tokenize_text(fname, vocab=None, invalid_label=0, start_label=1, sep_punctuation=True):
    lines = unidecode(open(fname).read().decode('utf-8')).split('\n')
    lines = [x for x in lines if x]
    lines = map(lambda x: re.findall(r"\w+|[^\w\s]", x, re.UNICODE), lines)    
    sentences, vocab = mx.rnn.encode_sentences(lines, vocab=vocab, invalid_label=invalid_label, start_label=start_label)
    return sentences, vocab

Dataset = namedtuple(
    'Dataset', 
    ['src_sent', 'src_vocab', 'inv_src_vocab', 'targ_sent', 'targ_vocab', 'inv_targ_vocab'])

def invert_dict(d):
    return {v: k for k, v in d.iteritems()}


def get_data(src_path, targ_path, start_label=1, invalid_label=0, pad_symbol='<PAD>'):
    src_sent, src_vocab = tokenize_text(src_path, start_label=start_label,
                                invalid_label=invalid_label)
    
    src_vocab[pad_symbol] = invalid_label
    inv_src_vocab = invert_dict(src_vocab)

    targ_sent, targ_vocab = tokenize_text(targ_path, start_label=start_label, #new_start+1,
                                          invalid_label=invalid_label)
    
    targ_vocab[pad_symbol] = invalid_label
    inv_targ_vocab = invert_dict(targ_vocab)
    
    return Dataset(
        src_sent=src_sent, src_vocab=src_vocab, inv_src_vocab=inv_src_vocab,
        targ_sent=targ_sent, targ_vocab=targ_vocab, inv_targ_vocab=inv_targ_vocab)

In [4]:
def persist_dataset(dataset, path):
    with open(path, 'wb+') as fileobj:
        pickle.dump(dataset, fileobj)
        
def load_dataset(path):
    with open(path, 'rb') as fileobj:
        return pickle.load(fileobj)

In [5]:
dataset = \
    get_data(
        src_path='./data/europarl-v7.es-en.en_vsmall',
        targ_path='./data/europarl-v7.es-en.es_vsmall',
        start_label=1,
        invalid_label=0
    )

In [13]:
class Seq2SeqIterator:    
    
    class TwoDBisect:
        def __init__(self, buckets):
            self.buckets = sorted(buckets, key=operator.itemgetter(0, 1))
            self.x, self.y = zip(*buckets)
            self.x, self.y = np.array(list(self.x)), np.array(list(self.y))

        def twod_bisect(self, source, target):    
            offset1 = np.searchsorted(self.x, len(source), side='left')
            offset2 = np.where(self.y[offset1:] >= len(target))[0]        
            return self.buckets[offset1 + offset2[0]] 
    
    def __init__(self, dataset, buckets=None, batch_size=32, max_sent_len=None):
        self.src_sent = dataset.src_sent
        if max_sent_len:
            self.src_sent = self.filter_long_sent(dataset.src_sent, max_sent_len)
        self.targ_sent = dataset.targ_sent
        if max_sent_len:
            self.targ_sent = self.filter_long_sent(dataset.targ_sent, max_sent_len)        
        self.src_vocab = dataset.src_vocab
        self.targ_vocab = dataset.targ_vocab
        self.inv_src_vocab = dataset.inv_src_vocab
        self.inv_targ_vocab = dataset.inv_targ_vocab
        self.buckets = buckets if buckets else self.gen_buckets(
            dataset, filter_smaller_counts_than=batch_size)
        self.bisect = TwoDBisect(self.buckets)
        self.max_sent_len = max_sent_len
    
    def group_lengths():
        tuples = []
        for src, targ in zip(src_sent, targ_sent):
            len_tup = bisect.twod_bisect(src, targ)
            rev_src = src[::-1] 
            tuples.append((src, targ, len_tup))
        sorted_tuples = sorted(tuples, key=operator.itemgetter(2))
        for i in groupby(sorted_tuples, lambda x: x[2])

    @staticmethod 
    def filter_long_sent(src_sent, targ_sent):
        return filter(lambda x: len(x[0]) <= max_len and len(x[1]) <= max_len, zip(src_sent, targ_sent))

    @staticmethod
    def gen_buckets(dataset, filter_smaller_counts_than=None, max_sent_len=60, min_sent_len=1):
        src_sent = dataset.src_sent
        targ_sent = dataset.targ_sent
        length_pairs = map(lambda x: (len(x[0]), len(x[1])), zip(src_sent, targ_sent))
        counts = list(Counter(length_pairs).items())
        c_sorted = sorted(counts, key=operator.itemgetter(0, 1))
        buckets = [i[0] for i in c_sorted if i[1] >= filter_smaller_counts_than and 
                   (max_sent_len is None or i[0][0] <= max_sent_len) and
                   (max_sent_len is None or i[0][1] <= max_sent_len) and
                   (min_sent_len is None or i[0][0] >= min_sent_len) and
                   (min_sent_len is None or i[0][1] >= min_sent_len)]
        return buckets

#    def reset(self):
#         self.curr_idx = 0
#         random.shuffle(self.idx)
#         for buck in self.data:
#             np.random.shuffle(buck)

#         self.nddata = []
#         self.ndlabel = []
#         for buck in self.data:
#             label = np.empty_like(buck)
#             label[:, :-1] = buck[:, 1:]
#             label[:, -1] = self.invalid_label
#             self.nddata.append(ndarray.array(buck, dtype=self.dtype))
#             self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

#     def next(self):
#         if self.curr_idx == len(self.idx):
#             raise StopIteration
#         i, j = self.idx[self.curr_idx]
#         self.curr_idx += 1

#         if self.major_axis == 1:
#             data = self.nddata[i][j:j+self.batch_size].T
#             label = self.ndlabel[i][j:j+self.batch_size].T
#         else:
#             data = self.nddata[i][j:j+self.batch_size]
#             label = self.ndlabel[i][j:j+self.batch_size]

#         return DataBatch([data], [label], pad=0,
#                          bucket_key=self.buckets[i],
#                          provide_data=[(self.data_name, data.shape)],
#                          provide_label=[(self.label_name, label.shape)])

SyntaxError: non-default argument follows default argument (<ipython-input-13-ad958118650b>, line 14)

In [7]:
i1 = Seq2SeqIterator(dataset)

In [8]:
src_sent = dataset.src_sent
targ_sent = dataset.targ_sent

sent_len = lambda x: map(lambda y: len(y), x)
max_len = lambda x: max(sent_len(x))
min_len = lambda x: min(sent_len(x))

min_len = min(min(sent_len(src_sent)), min(sent_len(targ_sent)))
# max_len = max(max(sent_len(src_sent)), max(sent_len(targ_sent)))

# min_len = min
max_len = 65
increment = 5

all_pairs = [(i, j) for i in range(
        min_len,max_len+increment,increment
    ) for j in range(
        min_len,max_len+increment,increment
    )]

In [9]:
i2 = Seq2SeqIterator(dataset, all_pairs)

In [11]:
i1.buckets

[(8, 13),
 (8, 18),
 (8, 20),
 (8, 26),
 (9, 20),
 (9, 22),
 (9, 28),
 (10, 12),
 (10, 13),
 (10, 15),
 (10, 17),
 (10, 21),
 (10, 26),
 (11, 12),
 (11, 13),
 (11, 14),
 (11, 15),
 (11, 20),
 (11, 21),
 (11, 22),
 (11, 23),
 (11, 24),
 (11, 25),
 (12, 9),
 (12, 11),
 (12, 13),
 (12, 14),
 (12, 15),
 (12, 17),
 (12, 18),
 (12, 19),
 (12, 20),
 (12, 21),
 (12, 22),
 (12, 23),
 (12, 24),
 (12, 25),
 (12, 27),
 (12, 29),
 (13, 12),
 (13, 14),
 (13, 15),
 (13, 16),
 (13, 18),
 (13, 19),
 (13, 20),
 (13, 21),
 (13, 22),
 (13, 23),
 (13, 24),
 (13, 28),
 (14, 9),
 (14, 10),
 (14, 12),
 (14, 13),
 (14, 14),
 (14, 15),
 (14, 17),
 (14, 18),
 (14, 19),
 (14, 20),
 (14, 21),
 (14, 22),
 (14, 24),
 (14, 25),
 (14, 27),
 (14, 28),
 (15, 12),
 (15, 13),
 (15, 15),
 (15, 17),
 (15, 18),
 (15, 19),
 (15, 20),
 (15, 21),
 (15, 22),
 (15, 23),
 (15, 24),
 (15, 25),
 (15, 26),
 (15, 27),
 (15, 28),
 (15, 29),
 (15, 34),
 (16, 10),
 (16, 11),
 (16, 12),
 (16, 13),
 (16, 14),
 (16, 15),
 (16, 16),
 (16, 17

In [12]:
i2.buckets

[(1, 1),
 (1, 6),
 (1, 11),
 (1, 16),
 (1, 21),
 (1, 26),
 (1, 31),
 (1, 36),
 (1, 41),
 (1, 46),
 (1, 51),
 (1, 56),
 (1, 61),
 (1, 66),
 (6, 1),
 (6, 6),
 (6, 11),
 (6, 16),
 (6, 21),
 (6, 26),
 (6, 31),
 (6, 36),
 (6, 41),
 (6, 46),
 (6, 51),
 (6, 56),
 (6, 61),
 (6, 66),
 (11, 1),
 (11, 6),
 (11, 11),
 (11, 16),
 (11, 21),
 (11, 26),
 (11, 31),
 (11, 36),
 (11, 41),
 (11, 46),
 (11, 51),
 (11, 56),
 (11, 61),
 (11, 66),
 (16, 1),
 (16, 6),
 (16, 11),
 (16, 16),
 (16, 21),
 (16, 26),
 (16, 31),
 (16, 36),
 (16, 41),
 (16, 46),
 (16, 51),
 (16, 56),
 (16, 61),
 (16, 66),
 (21, 1),
 (21, 6),
 (21, 11),
 (21, 16),
 (21, 21),
 (21, 26),
 (21, 31),
 (21, 36),
 (21, 41),
 (21, 46),
 (21, 51),
 (21, 56),
 (21, 61),
 (21, 66),
 (26, 1),
 (26, 6),
 (26, 11),
 (26, 16),
 (26, 21),
 (26, 26),
 (26, 31),
 (26, 36),
 (26, 41),
 (26, 46),
 (26, 51),
 (26, 56),
 (26, 61),
 (26, 66),
 (31, 1),
 (31, 6),
 (31, 11),
 (31, 16),
 (31, 21),
 (31, 26),
 (31, 31),
 (31, 36),
 (31, 41),
 (31, 46),
 (31, 51